In [251]:
import matplotlib.pyplot as plt 
#import scipy.misc
import numpy as np
import json
from scipy import misc
from scipy.misc import imread

import keras

from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D, Concatenate
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Add, merge, concatenate
from keras.models import Model
from keras.utils.layer_utils import print_summary
from keras.preprocessing.image import ImageDataGenerator

from keras import __version__
print(__version__)

2.0.5


In [19]:
train_folder = '/home/ubuntu/data/sar/experiment_crops_20170809/train'
valid_folder = '/home/ubuntu/data/sar/experiment_crops_20170809/validate'


In [278]:
main_shape = (50, 50 ,1)
aux_shape = (1,1)

main_input = Input(shape=main_shape, name='main_input')

aux_input = Input(shape=aux_shape, name='aux_input')
x2 = Dense(1, activation='relu')(aux_input)

num_classes = 3

x = Cropping2D(cropping=((10, 10), (10, 10)))(main_input)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)

added = keras.layers.merge([x, x2], mode=)

([x, x2])
#merged = keras.layers.merge([x, x2])

# oncatenate(axis=-1)
# Add([x, x2])
# # ([x, x2])


out = Dense(num_classes, activation='softmax')(added)
model = keras.models.Model(inputs=[main_input, aux_input], outputs=out)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:19: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


TypeError: <class 'keras.layers.merge.Add'> is not a Python function

In [257]:
model.output_shape

(None, 1, 3)

In [258]:
print main_input.shape
print aux_input.shape
model.summary()


(?, 50, 50, 1)
(?, 1, 1)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
cropping2d_37 (Cropping2D)       (None, 30, 30, 1)     0           main_input[0][0]                 
____________________________________________________________________________________________________
conv2d_106 (Conv2D)              (None, 30, 30, 32)    320         cropping2d_37[0][0]              
____________________________________________________________________________________________________
max_pooling2d_71 (MaxPooling2D)  (None, 15, 15, 32)    0           conv2d_106[0][0]                 
__________________________________________________________________

In [259]:
from keras.utils import plot_model
plot_model(model, to_file='/home/ubuntu/data/sar/experiment_crops_20170809/model.png')

In [260]:
import json

In [261]:
train_class = []           
train_filename = []
train_crops = []
train_feature = []

valid_class = []
valid_filename = []
valid_crops = []
valid_feature = []

train_class_desc = 'oil_and_gas_infrastructure'
train_class_array = [1,0,0]
with open('/home/ubuntu/data/sar/experiment_crops_20170809/train/oil_and_gas_infrastructure_train.json') as json_data:
    json_train_data = json.load(json_data)
    for id_, item in json_train_data.iteritems():
        train_filename.append(id_)
        train_feature.append(item['distance to land'])
        train_class.append(train_class_array)
        file_path = train_folder + '/' + train_class_desc + '/' + id_
        img  = imread(file_path)
        train_crops.append(img)

train_class_array = [0,1,0]
train_class_desc = 'turbine'  
with open('/home/ubuntu/data/sar/experiment_crops_20170809/train/turbine_train.json') as json_data:
    json_train_data = json.load(json_data)
    for id_, item in json_train_data.iteritems():
        train_filename.append(id_)
        train_feature.append(item['distance to land'])
        train_class.append(train_class_array)
        file_path = train_folder + '/' + train_class_desc + '/' + id_
        img  = imread(file_path)
        train_crops.append(img)
        
train_class_array = [0,0,1]
train_class_desc = 'other'  
with open('/home/ubuntu/data/sar/experiment_crops_20170809/train/other_train.json') as json_data:
    json_train_data = json.load(json_data)
    for id_, item in json_train_data.iteritems():
        train_filename.append(id_)
        train_feature.append(item['distance to land'])
        train_class.append(train_class_array)
        file_path = train_folder + '/' + train_class_desc + '/' + id_
        img  = imread(file_path)
        train_crops.append(img)
    
valid_class_array = [1,0,0]
valid_class_desc = 'oil_and_gas_infrastructure'  
with open('/home/ubuntu/data/sar/experiment_crops_20170809/validate/oil_and_gas_infrastructure_validate.json') as json_data:
    json_validation_data = json.load(json_data)
    for id_, item in json_validation_data.iteritems():
        valid_filename.append(id_)
        valid_feature.append(item['distance to land'])
        valid_class.append(valid_class_array)
        file_path = valid_folder + '/' + valid_class_desc + '/' + id_
        img  = imread(file_path)
        valid_crops.append(img)
        
valid_class_array = [0,1,0]
valid_class_desc = 'turbine'  
with open('/home/ubuntu/data/sar/experiment_crops_20170809/validate/turbine_validate.json') as json_data:
    json_validation_data = json.load(json_data)
    for id_, item in json_validation_data.iteritems():
        valid_filename.append(id_)
        valid_feature.append(item['distance to land'])
        valid_class.append(valid_class_array)
        file_path = valid_folder + '/' + valid_class_desc + '/' + id_
        img  = imread(file_path)
        valid_crops.append(img)

valid_class_array = [0,0,1]
valid_class_desc = 'other'  
with open('/home/ubuntu/data/sar/experiment_crops_20170809/validate/other_validate.json') as json_data:
    json_validation_data = json.load(json_data)
    for id_, item in json_validation_data.iteritems():
        valid_filename.append(id_)
        valid_feature.append(item['distance to land'])
        valid_class.append(valid_class_array)
        file_path = valid_folder + '/' + valid_class_desc + '/' + id_
        img  = imread(file_path)
        valid_crops.append(img)
    


In [262]:
print "training crops:", len(train_filename)
print "training features:", len(train_feature)
print "validation crops:", len(valid_filename)
print "validation features:", len(valid_feature)



training crops: 2000
training features: 2000
validation crops: 500
validation features: 500


In [263]:
np_train_class = np.array(train_class)
np_train_filename = np.array(train_filename)
np_train_crops = np.array(train_crops)
np_train_feature = np.array(train_feature)

np_valid_class = np.array(valid_class)
np_valid_filename = np.array(valid_filename)
np_valid_crops = np.array(valid_crops)
np_valid_feature = np.array(valid_feature)




In [264]:
print np_train_crops.shape
print np_train_feature.shape
print np_train_class.shape

print np_valid_crops.shape
print np_valid_feature.shape
print np_valid_class.shape


(2000, 50, 50)
(2000,)
(2000, 3)
(500, 50, 50)
(500,)
(500, 3)


In [265]:
# got here!

test = np.expand_dims(np_train_crops, axis=3)
test_aux = np.expand_dims(np_train_feature, axis=1)
test_aux = np.expand_dims(test_aux, axis=2)


print test.shape

print test_aux.shape



model.train_on_batch([test, test_aux], np_train_class)

# model.fit([np_train_crops, np_train_feature], np_train_class, batch_size=10, nb_epoch=1,
#           validation_data=([np_valid_crops, np_valid_feature], np_valid_class))



(2000, 50, 50, 1)
(2000, 1, 1)


ValueError: Error when checking target: expected dense_54 to have 3 dimensions, but got array with shape (2000, 3)

In [266]:
model.optimizer.lr = 0.0001

In [267]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1000,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=60)

NameError: name 'train_generator' is not defined

In [ ]:

# I think I found the bit that Andrew did.. not sure if we should do it that way but the links are here:    
""" Based heavily on:
https://shaoanlu.wordpress.com/2017/04/10/a-simple-pseudo-labeling-function-implementation-in-keras/
https://github.com/shaoanlu/dogs-vs-cats-redux/blob/master/res50_incepV3_Xcept.ipynb
https://github.com/asmith26/learningWithKaggle/blob/master/as/the-nature-conservancy-fisheries-monitoring_v1-keras2-pseudo_labelling.ipynb
"""

# I would try doing the .fit which is all the data as you suggested, you need to make sure the data is ordered correctly, 
# then pass it into the following. 
# I think this needs to have the shapes ([(num_images, 140, 140, 1), (num_images, 1)], (num_images, 1))
model.fit([img, features], label)         